In [1]:
#Import thư viện 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from underthesea import word_tokenize, pos_tag, sent_tokenize
import warnings
from gensim import corpora, models, similarities
import re
import regex
from pandas_profiling import ProfileReport
from wordcloud import WordCloud

C:\Users\Admin\AppData\Local\Temp\ipykernel_13116\3953600914.py:12: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


In [2]:
# Đọc dữ liệu
df2=pd.read_csv('San_pham.csv')

In [3]:
# Xem vài dòng
df2.head(10)

,ma_san_pham,ten_san_pham,gia_ban,gia_goc,phan_loai,mo_ta,diem_trung_binh
0,318900012,Nước Hoa Hồng Klairs Không Mùi Cho Da Nhạy Cảm...,209000,435000.0,2x180ml\n180ml\nKhông Mùi\nCó Mùi Hương,Nước Hoa Hồng Klairs Supple Preparation là dòn...,4.8
1,205100137,"Nước Tẩy Trang L'Oreal Tươi Mát Cho Da Dầu, Hỗ...",147000,229000.0,2x400ml\n95ml\n400ml\nLàm Sạch Sâu\nTươi Mát D...,Nước Tẩy Trang L'Oréal là dòng sản phẩm tẩy tr...,4.7
2,422208973,Sữa Rửa Mặt CeraVe Sạch Sâu Cho Da Thường Đến ...,343000,455000.0,88ml\n236ml\n473ml\nDa khô/Hỗn hợp khô\nDa dầu...,Sữa Rửa Mặt Cerave Sạch Sâu là sản phẩm sữa rử...,4.9
3,204900013,Kem Chống Nắng La Roche-Posay Kiểm Soát Dầu SP...,377000,560000.0,2x50ml\n50ml,Kem chống nắng giúp bảo vệ da khỏi tia UVB & U...,4.6
4,253900006,Kem Chống Nắng Skin1004 Cho Da Nhạy Cảm SPF 50...,210000,445000.0,20ml\n50ml,Kem Chống Nắng Skin1004 Cho Da Nhạy Cảm là sản...,4.6
5,200400003,Nước Tẩy Trang Bioderma Dành Cho Da Nhạy Cảm 5...,312000,525000.0,"Combo 1000ml\n100ml\n250ml\n500ml\nDa Dầu, Da ...",Nước Tẩy Trang Bioderma bao gồm 2 phân loại: \...,4.9
6,422218358,Sữa Chống Nắng Anessa Dưỡng Da Kiềm Dầu 60ml (...,533000,715000.0,2x20ml\n20ml\n60ml\n60ml+20ml\n60ml+2x20ml,Sữa Chống Nắng Anessa Perfect UV Sunscreen Ski...,4.9
7,200400004,Nước Tẩy Trang Bioderma Dành Cho Da Dầu & Hỗn ...,334000,525000.0,"Combo 1000ml\n100ml\n250ml\n500ml\nDa Dầu, Da ...",Nước Tẩy Trang Bioderma bao gồm 2 phân loại: \...,4.9
8,422210400,Kem Chống Nắng L'Oreal Paris X20 Thoáng Da Mỏn...,246000,379000.0,2x50ml\n30ml\n50ml\nx20 Mỏng Nhẹ\nDưỡng Ẩm\nNâ...,UV Defender SPF 50+ PA++++ là dòng sản phẩm ch...,4.1
9,205100146,Nước Tẩy Trang L'Oreal Làm Sạch Sâu Trang Điểm...,174000,269000.0,2x400ml\n95ml\n400ml\nLàm Sạch Sâu\nTươi Mát D...,Nước Tẩy Trang L'Oréal là dòng sản phẩm tẩy tr...,4.7


In [4]:
# Xem thông tin
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ma_san_pham      1200 non-null   int64  
 1   ten_san_pham     1200 non-null   object 
 2   gia_ban          1200 non-null   int64  
 3   gia_goc          1186 non-null   float64
 4   phan_loai        678 non-null    object 
 5   mo_ta            1200 non-null   object 
 6   diem_trung_binh  1200 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 65.8+ KB


In [5]:
# Kiểm tra dữ liệu bị thiếu
df2.isnull().sum()

ma_san_pham          0
ten_san_pham         0
gia_ban              0
gia_goc             14
phan_loai          522
mo_ta                0
diem_trung_binh      0
dtype: int64

In [6]:
# Kiểm tra số lượng giá trị thiếu trong từng cột
missing_values = df2[['gia_goc', 'phan_loai']].isnull().sum()
total_rows = len(df2)

print("Số lượng giá trị thiếu:")
print(missing_values)

# Tính tỷ lệ giá trị thiếu
missing_percentage = (missing_values / total_rows) * 100
print("\nTỷ lệ giá trị thiếu (%):")
print(missing_percentage)


Số lượng giá trị thiếu:
gia_goc       14
phan_loai    522
dtype: int64

Tỷ lệ giá trị thiếu (%):
gia_goc       1.166667
phan_loai    43.500000
dtype: float64


In [7]:
# Thay thế giá trị thiếu trong cột 'gia_goc' bằng trung bình
mean_value = df2['gia_goc'].mean()
df2['gia_goc'].fillna(mean_value, inplace=True)

In [8]:

for idx, row in df2[df2['phan_loai'].isnull()].iterrows():
    try:
        # Chuẩn bị từ khóa tìm kiếm
        if isinstance(row['ten_san_pham'], str):
            search_term = re.escape(row['ten_san_pham'].split()[0])
            
            # Tìm các sản phẩm tương tự (vô hiệu hóa regex nếu cần)
            similar_products = df2[df2['ten_san_pham'].str.contains(search_term, regex=True, na=False)]
            
            # Điền giá trị nếu tìm thấy sản phẩm tương tự
            if not similar_products['phan_loai'].mode().empty:
                df2.at[idx, 'phan_loai'] = similar_products['phan_loai'].mode()[0]
            else:
                df2.at[idx, 'phan_loai'] = "Không xác định"
        else:
            df2.at[idx, 'phan_loai'] = "Không xác định"
    except Exception as e:
        print(f"Lỗi tại index {idx}: {e}")


In [9]:
# Kiểm tra lại
df2.isnull().sum()

ma_san_pham        0
ten_san_pham       0
gia_ban            0
gia_goc            0
phan_loai          0
mo_ta              0
diem_trung_binh    0
dtype: int64

In [10]:
# Kiểm tra dữ liệu trùng lặp
df2.duplicated().any()

False

In [11]:
# In ra các cột
df2.columns.to_list()


['ma_san_pham',
 'ten_san_pham',
 'gia_ban',
 'gia_goc',
 'phan_loai',
 'mo_ta',
 'diem_trung_binh']

In [12]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ma_san_pham      1200 non-null   int64  
 1   ten_san_pham     1200 non-null   object 
 2   gia_ban          1200 non-null   int64  
 3   gia_goc          1200 non-null   float64
 4   phan_loai        1200 non-null   object 
 5   mo_ta            1200 non-null   object 
 6   diem_trung_binh  1200 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 65.8+ KB


# Chuẩn hoá dữ liệu

In [13]:
# Khám phá dữ liệu
# Dùng pandas_profiling sau đó xuất báo cáo (tên:san_pham_report.html) để xem chi tiết
profile = ProfileReport(df2, title="San Pham Data Profiling Report")
profile.to_file("san_pham_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

c:\Program Files\Python39\lib\site-packages\ydata_profiling\model\pandas\discretize_pandas.py:52: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  discretized_df.loc[:, column] = self._discretize_column(


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# Chuẩn hóa text
<a> Chuẩn hoá cột mo_ta

In [14]:
def load_stop_words(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            stop_words = file.read()
        return stop_words.split('\n')
    except Exception as e:
        print(f"Error loading stop words: {e}")
        return []

# Sử dụng hàm
STOP_WORD_FILE = 'files/vietnamese-stopwords.txt'
stop_words = load_stop_words(STOP_WORD_FILE)


# Underthesea
#### word_tokenize, pos_tag, sent_tokenize

In [15]:
# Lựa chọn các thông tin cần thiết để đưa vào phần content (giúp đề xuất nội dung)
df2['Content'] = df2['mo_ta'].apply(lambda x: ' '.join(x.split()[:500]))

In [16]:
# Xem vài dòng
df2['Content'].head(10)

0    Nước Hoa Hồng Klairs Supple Preparation là dòn...
1    Nước Tẩy Trang L'Oréal là dòng sản phẩm tẩy tr...
2    Sữa Rửa Mặt Cerave Sạch Sâu là sản phẩm sữa rử...
3    Kem chống nắng giúp bảo vệ da khỏi tia UVB & U...
4    Kem Chống Nắng Skin1004 Cho Da Nhạy Cảm là sản...
5    Nước Tẩy Trang Bioderma bao gồm 2 phân loại: N...
6    Sữa Chống Nắng Anessa Perfect UV Sunscreen Ski...
7    Nước Tẩy Trang Bioderma bao gồm 2 phân loại: N...
8    UV Defender SPF 50+ PA++++ là dòng sản phẩm ch...
9    Nước Tẩy Trang L'Oréal là dòng sản phẩm tẩy tr...
Name: Content, dtype: object

In [17]:
# Word_tokenize:  tách văn bản thành các từ có ý nghĩa
df2["Content_wt"]=df2["Content"].apply(lambda x: word_tokenize(x, format="text"))

In [18]:
#Xem vài dòng
df2[["Content", "Content_wt"]].head(10)

,Content,Content_wt
0,Nước Hoa Hồng Klairs Supple Preparation là dòn...,Nước Hoa_Hồng Klairs_Supple Preparation là dòn...
1,Nước Tẩy Trang L'Oréal là dòng sản phẩm tẩy tr...,Nước Tẩy_Trang L'Oréal là dòng sản_phẩm tẩy_tr...
2,Sữa Rửa Mặt Cerave Sạch Sâu là sản phẩm sữa rử...,Sữa Rửa Mặt_Cerave Sạch_Sâu là sản_phẩm sữa_rử...
3,Kem chống nắng giúp bảo vệ da khỏi tia UVB & U...,Kem chống nắng giúp bảo_vệ da khỏi tia UVB & U...
4,Kem Chống Nắng Skin1004 Cho Da Nhạy Cảm là sản...,Kem Chống Nắng_Skin1004 Cho_Da Nhạy_Cảm là sản...
5,Nước Tẩy Trang Bioderma bao gồm 2 phân loại: N...,Nước Tẩy_Trang Bioderma bao_gồm 2 phân_loại : ...
6,Sữa Chống Nắng Anessa Perfect UV Sunscreen Ski...,Sữa Chống Nắng Anessa_Perfect UV_Sunscreen Ski...
7,Nước Tẩy Trang Bioderma bao gồm 2 phân loại: N...,Nước Tẩy_Trang Bioderma bao_gồm 2 phân_loại : ...
8,UV Defender SPF 50+ PA++++ là dòng sản phẩm ch...,UV Defender SPF 50 + PA + + + + là dòng sản_ph...
9,Nước Tẩy Trang L'Oréal là dòng sản phẩm tẩy tr...,Nước Tẩy_Trang L'Oréal là dòng sản_phẩm tẩy_tr...


# Làm sạch thẻ html

In [19]:
# Hàm làm sạch HTML và các ký tự đặc biệt
def clean_html(text):
    if pd.isna(text):
        return text  # Giữ nguyên nếu là NaN
    # Loại bỏ thẻ HTML
    text = re.sub(r'<[^>]+>', '', text)
    # Loại bỏ ký tự xuống dòng và khoảng trắng thừa
    text = re.sub(r'\s+', ' ', text).strip()
    # Chuyển về chữ thường
    text = text.lower()
    return text

In [20]:
# Áp dụng hàm vào cột mô tả sản phẩm (ví dụ: 'Content')
df2['Content_wt'] = df2['Content'].apply(clean_html)

In [21]:

# Thay đổi tên cột trực tiếp
df2.rename(columns={'ma_san_pham': 'movieId', 'diem_trung_binh': 'rating'}, inplace=True)

In [23]:
df2.head(2)

,movieId,ten_san_pham,gia_ban,gia_goc,phan_loai,mo_ta,rating,Content,Content_wt
0,318900012,Nước Hoa Hồng Klairs Không Mùi Cho Da Nhạy Cảm...,209000,435000.0,2x180ml\n180ml\nKhông Mùi\nCó Mùi Hương,Nước Hoa Hồng Klairs Supple Preparation là dòn...,4.8,Nước Hoa Hồng Klairs Supple Preparation là dòn...,nước hoa hồng klairs supple preparation là dòn...
1,205100137,"Nước Tẩy Trang L'Oreal Tươi Mát Cho Da Dầu, Hỗ...",147000,229000.0,2x400ml\n95ml\n400ml\nLàm Sạch Sâu\nTươi Mát D...,Nước Tẩy Trang L'Oréal là dòng sản phẩm tẩy tr...,4.7,Nước Tẩy Trang L'Oréal là dòng sản phẩm tẩy tr...,nước tẩy trang l'oréal là dòng sản phẩm tẩy tr...


In [24]:
# Lưu file sau khi xử lí
df2.to_csv('San_pham_2xuly.csv', index=False)
#xem vài dòng sau khi xử lí
df2.head(10)

,movieId,ten_san_pham,gia_ban,gia_goc,phan_loai,mo_ta,rating,Content,Content_wt
0,318900012,Nước Hoa Hồng Klairs Không Mùi Cho Da Nhạy Cảm...,209000,435000.0,2x180ml\n180ml\nKhông Mùi\nCó Mùi Hương,Nước Hoa Hồng Klairs Supple Preparation là dòn...,4.8,Nước Hoa Hồng Klairs Supple Preparation là dòn...,nước hoa hồng klairs supple preparation là dòn...
1,205100137,"Nước Tẩy Trang L'Oreal Tươi Mát Cho Da Dầu, Hỗ...",147000,229000.0,2x400ml\n95ml\n400ml\nLàm Sạch Sâu\nTươi Mát D...,Nước Tẩy Trang L'Oréal là dòng sản phẩm tẩy tr...,4.7,Nước Tẩy Trang L'Oréal là dòng sản phẩm tẩy tr...,nước tẩy trang l'oréal là dòng sản phẩm tẩy tr...
2,422208973,Sữa Rửa Mặt CeraVe Sạch Sâu Cho Da Thường Đến ...,343000,455000.0,88ml\n236ml\n473ml\nDa khô/Hỗn hợp khô\nDa dầu...,Sữa Rửa Mặt Cerave Sạch Sâu là sản phẩm sữa rử...,4.9,Sữa Rửa Mặt Cerave Sạch Sâu là sản phẩm sữa rử...,sữa rửa mặt cerave sạch sâu là sản phẩm sữa rử...
3,204900013,Kem Chống Nắng La Roche-Posay Kiểm Soát Dầu SP...,377000,560000.0,2x50ml\n50ml,Kem chống nắng giúp bảo vệ da khỏi tia UVB & U...,4.6,Kem chống nắng giúp bảo vệ da khỏi tia UVB & U...,kem chống nắng giúp bảo vệ da khỏi tia uvb & u...
4,253900006,Kem Chống Nắng Skin1004 Cho Da Nhạy Cảm SPF 50...,210000,445000.0,20ml\n50ml,Kem Chống Nắng Skin1004 Cho Da Nhạy Cảm là sản...,4.6,Kem Chống Nắng Skin1004 Cho Da Nhạy Cảm là sản...,kem chống nắng skin1004 cho da nhạy cảm là sản...
5,200400003,Nước Tẩy Trang Bioderma Dành Cho Da Nhạy Cảm 5...,312000,525000.0,"Combo 1000ml\n100ml\n250ml\n500ml\nDa Dầu, Da ...",Nước Tẩy Trang Bioderma bao gồm 2 phân loại: \...,4.9,Nước Tẩy Trang Bioderma bao gồm 2 phân loại: N...,nước tẩy trang bioderma bao gồm 2 phân loại: n...
6,422218358,Sữa Chống Nắng Anessa Dưỡng Da Kiềm Dầu 60ml (...,533000,715000.0,2x20ml\n20ml\n60ml\n60ml+20ml\n60ml+2x20ml,Sữa Chống Nắng Anessa Perfect UV Sunscreen Ski...,4.9,Sữa Chống Nắng Anessa Perfect UV Sunscreen Ski...,sữa chống nắng anessa perfect uv sunscreen ski...
7,200400004,Nước Tẩy Trang Bioderma Dành Cho Da Dầu & Hỗn ...,334000,525000.0,"Combo 1000ml\n100ml\n250ml\n500ml\nDa Dầu, Da ...",Nước Tẩy Trang Bioderma bao gồm 2 phân loại: \...,4.9,Nước Tẩy Trang Bioderma bao gồm 2 phân loại: N...,nước tẩy trang bioderma bao gồm 2 phân loại: n...
8,422210400,Kem Chống Nắng L'Oreal Paris X20 Thoáng Da Mỏn...,246000,379000.0,2x50ml\n30ml\n50ml\nx20 Mỏng Nhẹ\nDưỡng Ẩm\nNâ...,UV Defender SPF 50+ PA++++ là dòng sản phẩm ch...,4.1,UV Defender SPF 50+ PA++++ là dòng sản phẩm ch...,uv defender spf 50+ pa++++ là dòng sản phẩm ch...
9,205100146,Nước Tẩy Trang L'Oreal Làm Sạch Sâu Trang Điểm...,174000,269000.0,2x400ml\n95ml\n400ml\nLàm Sạch Sâu\nTươi Mát D...,Nước Tẩy Trang L'Oréal là dòng sản phẩm tẩy tr...,4.7,Nước Tẩy Trang L'Oréal là dòng sản phẩm tẩy tr...,nước tẩy trang l'oréal là dòng sản phẩm tẩy tr...


# Xử lý dữ liệu thêm tập khách hàng sử dụng nếu cần thiết 

In [25]:
df3=pd.read_csv('Khach_hang.csv')

In [26]:
# Xem vài dòng
df3.head(50)

,ma_khach_hang,ho_ten
0,1,Hoàng Anh Trần
1,2,Nguyễn Lương Nhi Phương
2,3,Nguyễn Thu Trang
3,4,ANH THƯ
4,5,DIỆU ÁNH
5,6,Linh Đỗ
6,7,Thanh Trúc
7,8,Phạm Tiến Anh
8,9,Phan Thị Quý
9,10,Vn Team


In [27]:
#Kiểm tra trùng lặp
df3.duplicated().any()

False

In [28]:
# Chuẩn hóa cột 'ho_ten'
def standardize_name(name):
    if pd.isna(name):  # Xử lý giá trị NaN
        return name
    name = name.strip()  # Xóa khoảng trắng thừa
    name = " ".join(name.split())  # Xóa khoảng trắng thừa giữa các từ
    name = name.title()  # Viết hoa chữ cái đầu mỗi từ
    return name

df3['ho_ten'] = df3['ho_ten'].apply(standardize_name)
df3.head(30)

,ma_khach_hang,ho_ten
0,1,Hoàng Anh Trần
1,2,Nguyễn Lương Nhi Phương
2,3,Nguyễn Thu Trang
3,4,Anh Thư
4,5,Diệu Ánh
5,6,Linh Đỗ
6,7,Thanh Trúc
7,8,Phạm Tiến Anh
8,9,Phan Thị Quý
9,10,Vn Team


In [29]:
# Kiểm tra dữ liệu thiếu
df3.isnull().sum()

ma_khach_hang    0
ho_ten           1
dtype: int64

In [30]:
# Điền giá trị thay thế vào cột ho_ten
df3['ho_ten'].fillna('Không Rõ', inplace=True)

In [31]:
# Kiểm tra lại dữ liệu
df3.isnull().sum()

ma_khach_hang    0
ho_ten           0
dtype: int64

In [32]:
#  Kiểm tra tính duy nhất của 'ma_khach_hang'
if not df3['ma_khach_hang'].is_unique:
    print("Cảnh báo: Mã khách hàng bị trùng!")

In [33]:
# Thay đổi tên cột trực tiếp
df3.rename(columns={'ma_khach_hang': 'userId'}, inplace=True)

In [34]:
# Xem sau khi đổi tên cột
df3.head(2)

,userId,ho_ten
0,1,Hoàng Anh Trần
1,2,Nguyễn Lương Nhi Phương


In [37]:
# Lưu file sau khi xử lí
df3.to_csv("Khach_hang_2xuly.csv", index=False)


In [38]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4737 entries, 0 to 4736
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   userId  4737 non-null   int64 
 1   ho_ten  4737 non-null   object
dtypes: int64(1), object(1)
memory usage: 74.1+ KB


# Tạo 1 file có tên : movielens_ratings

In [ ]:
df2_ = pd.DataFrame(df2)
df3_ = pd.DataFrame(df3)

# Chỉ lấy cột 'movieId','rating' từ df2
df2_selected = df2[['movieId','rating']]

# Chỉ lấy cột 'userId' từ df3
df3_selected = df3[['userId']]

# Tạo DataFrame mới bằng cách ghép các cột đã chọn
df_new = pd.concat([df2_selected, df3_selected], axis=1)

print("DataFrame mới sau khi kết hợp:")
print(df_new.head(10))

DataFrame mới sau khi kết hợp:
       movieId  rating  userId
0  318900012.0     4.8       1
1  205100137.0     4.7       2
2  422208973.0     4.9       3
3  204900013.0     4.6       4
4  253900006.0     4.6       5
5  200400003.0     4.9       6
6  422218358.0     4.9       7
7  200400004.0     4.9       8
8  422210400.0     4.1       9
9  205100146.0     4.7      10


In [44]:
# Lưu file sau khi xử lí
df_new.to_csv("movielens_ratings.csv", index=False)